# pilot test3
## - main protocol test
## - 32ch
## - ch1: empty, ch33: empty --> 둘다 삭제
## - txt2mat

### 20220126 작성 시작

In [1]:
import os 
import numpy as np
import natsort
import pandas as pd
from tqdm.notebook import tqdm, trange
import scipy.io as spio
%matplotlib inline

In [2]:
default_path = 'E:\\B2X\\2차년도\\3차 pilot\\subject_data'
default_path

'E:\\B2X\\2차년도\\3차 pilot\\subject_data'

## subject list 확보

In [3]:
def OnlyFolder(path):
    # return only folder, not file
    # path: default_path which contains subject folders
    
    dir_list = os.listdir(path) # get all directory below the path
    output = []
    for i in range(len(dir_list)):
        temp_path = os.path.join(path, dir_list[i])
        if os.path.isdir(temp_path) == True:
            output.append(dir_list[i])
    return output

In [4]:
total_subject_list = OnlyFolder(default_path)
total_subject_list = natsort.natsorted(total_subject_list) # 번호 순서대로 정렬
total_subject_list

['01_KDH',
 '02_HHB',
 '03_PSR',
 '04_CJK',
 '05_CKM',
 '06_KDH',
 '07_PSR',
 '08_HHB',
 '09_CKM',
 '10_CSJ',
 '11_KMS',
 '12_CJK',
 '13_CSJ',
 '14_KMS',
 '15_LJH',
 '16_PBJ',
 '17_PBJ',
 '18_SYH',
 '19_SYH',
 '20_LJH']

## 분석할 subject 선택

In [40]:
# XX_HKD 의 XX를 그대로 입력 
# ex) 05_HKD --> 5
choose_sub = range(10,21)
sub_list = []
for sub_num in range(len(choose_sub)):
   sub_list.append(total_subject_list[choose_sub[sub_num]-1])
    
print('Target Subjects: {}'.format(sub_list))

Target Subjects: ['10_CSJ', '11_KMS', '12_CJK', '13_CSJ', '14_KMS', '15_LJH', '16_PBJ', '17_PBJ', '18_SYH', '19_SYH', '20_LJH']


In [41]:
def SelectTxtFiles(file_list):
    # file_list 중, 'txt' 문자열을 포함한 것만 남김
    # file_list : 검사하고자 하는 file 이름이 담긴 list
    print("\nTest... if txt or not")
    result = []
    for i in range(len(file_list)):
        test = file_list[i][-3:]
        if test == 'txt':
            result.append(file_list[i])
    print('Filtered txt files: {}'.format(result))
    return result


def TxtRead(path, filename):
    # txt 파일을 line별로 읽어서 list로 반환
    # path: 파일이 저장된 경로
    # filname : 파일 이름

    file_path = path + '\\' + filename
    
    result = []
    num = 0
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            data = line.strip().split()
            result.append(data)
    return result
            



In [42]:
for sub_num in tqdm(range(len(choose_sub)), desc='Total Processing'):
    
    print('------------------------------------')
    print(sub_list[sub_num])
    # 현재 sub_num에 해당하는 subject의 txt 파일이 존재하는 경로
    subject_txt_path = default_path + '\\' + sub_list[sub_num] + '\\EEG'
    
    # txt 경로에서, 확장자명이 txt인 파일들만 선택
    txt_list = SelectTxtFiles(os.listdir(subject_txt_path))
    print('\nData Extraction Start ...')
    
    # 5개의 txt 파일에 대해서 각각 data를 추출하고, mat파일로 저장
    for stim_num in tqdm(range(len(txt_list)), desc=sub_list[sub_num] + ' Processing'):
        print('{}th txt'.format(stim_num + 1), end='.. ')
        
        temp_data = TxtRead(subject_txt_path, txt_list[stim_num])
        header = temp_data[:3] # header 성분
        non_header = temp_data[3:] # non-header --> EEG data
    
        # non_header의 마지막 row에는 빈 배열이 존재하기 때문에, numpy로 전환하면서 오류가 발생함
        # --> 마지막 row를 삭제
        if len(non_header[len(non_header)-1]) == 0:
            del non_header[len(non_header)-1] # list의 요소 제거 방법인 del을 사용
            # non_header list를 numpy array로 변환
            non_header = np.array(non_header, dtype=float)
        non_header_df = pd.DataFrame(non_header)
        
        # ch34 유무에 따른 if 문
        if non_header_df.shape[1] == 34:
            # ch34기 존재하면 이는 flag를 저장하기 위한 채널이므로 삭제하여 준다.
            # ch1은 빈배열이므로 index 0을 이용하여 제거
            # ch34는index 33을 이용하여 제거
            non_header_df = non_header_df.drop([0, 33], axis=1)
        else:
            # ch34가 존재하지 않으면, 빈 채널인 ch1만 index 0을 이용하여 제거            
            non_header_df = non_header_df.drop(0, axis=1)
            
        print(" Shape: {}, Size: {}".format(non_header_df.shape, non_header_df.size), end=' --> Save...')
        
        data = np.array(non_header_df)
        save_as = os.path.join(subject_txt_path, txt_list[stim_num][:-3]+'mat')
        spio.savemat(save_as, mdict={'data': data})
        print("done !")

Total Processing:   0%|          | 0/11 [00:00<?, ?it/s]

------------------------------------
10_CSJ

Test... if txt or not
Filtered txt files: ['s0.txt', 's1_N100.txt', 's2_S100.txt', 's3_N020.txt', 's4_N020.txt']

Data Extraction Start ...


10_CSJ Processing:   0%|          | 0/5 [00:00<?, ?it/s]

1th txt..  Shape: (561536, 32), Size: 17969152 --> Save...done !
2th txt..  Shape: (556416, 32), Size: 17805312 --> Save...done !
3th txt..  Shape: (559296, 32), Size: 17897472 --> Save...done !
4th txt..  Shape: (556992, 32), Size: 17823744 --> Save...done !
5th txt..  Shape: (556416, 32), Size: 17805312 --> Save...done !
------------------------------------
11_KMS

Test... if txt or not
Filtered txt files: ['s0.txt', 's1_S100.txt', 's2_N100.txt', 's3_S020.txt', 's4_N020.txt']

Data Extraction Start ...


11_KMS Processing:   0%|          | 0/5 [00:00<?, ?it/s]

...

1th txt..  Shape: (558720, 32), Size: 17879040 --> Save...done !
2th txt..  Shape: (556672, 32), Size: 17813504 --> Save...done !
3th txt..  Shape: (557760, 32), Size: 17848320 --> Save...done !
4th txt..  Shape: (557312, 32), Size: 17833984 --> Save...done !
5th txt..  Shape: (556800, 32), Size: 17817600 --> Save...done !
------------------------------------
19_SYH

Test... if txt or not
Filtered txt files: ['s0.txt', 's1_N100.txt', 's2_S100.txt', 's3_N020.txt', 's4_S020.txt']

Data Extraction Start ...


19_SYH Processing:   0%|          | 0/5 [00:00<?, ?it/s]

1th txt..  Shape: (558272, 32), Size: 17864704 --> Save...done !
2th txt..  Shape: (594240, 32), Size: 19015680 --> Save...done !
3th txt..  Shape: (558528, 32), Size: 17872896 --> Save...done !
4th txt..  Shape: (558464, 32), Size: 17870848 --> Save...done !
5th txt..  Shape: (558336, 32), Size: 17866752 --> Save...done !
------------------------------------
20_LJH

Test... if txt or not
Filtered txt files: ['s0.txt', 's1_S100.txt', 's2_N020.txt', 's3_N100.txt', 's4_S020.txt']

Data Extraction Start ...


20_LJH Processing:   0%|          | 0/5 [00:00<?, ?it/s]

1th txt..  Shape: (556032, 32), Size: 17793024 --> Save...done !
2th txt..  Shape: (557824, 32), Size: 17850368 --> Save...done !
3th txt..  Shape: (557632, 32), Size: 17844224 --> Save...done !
4th txt..  Shape: (557952, 32), Size: 17854464 --> Save...done !
5th txt..  Shape: (555968, 32), Size: 17790976 --> Save...done !


In [19]:
k = []

In [21]:
len(k)

0

In [32]:
k = np.arange(0,6)
print(type(k))

<class 'numpy.ndarray'>


In [33]:
t = range(0,5)

In [34]:
t[0]

0

In [35]:
t[1]

1

In [36]:
t[4]

4

In [37]:
t[5]

IndexError: range object index out of range